In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings("ignore")

In [ ]:
ques_df = pd.read_csv('/content/drive/MyDrive/NLP/duplicate sent proj/ques_df_1.csv')
ques_df.head()

,question1,question2
0,what is the best java program to implement stack,what are some good java programs to practice with
1,can donald trump still win the election,does donald trump has chamces to win us elections
2,how can one be less of an introvert,can one be an extrovert but also an introvert
3,people should not have the right to commit sui...,why do not we have a right to commit suicide
4,how can i make money as a 13 year old,how can i make money as a thirteen year old


In [ ]:

final_df = pd.read_csv('/content/drive/MyDrive/NLP/duplicate sent proj/final_df_1.csv')
final_df.head()

,is_duplicate,q1_len,q2_len,q1_word,q2_word,word_common,total_words,word_share,cwc_min,cwc_max,...,ctc_max,last_word_equal,first_word_equal,abs_leng_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
0,0,48,49,9,9,3,18,0.166667,0.199996,0.249994,...,0.333330,0,1,1,9.0,0.265306,58,58,54,62
1,1,39,49,7,9,3,16,0.187500,0.599988,0.499992,...,0.333330,0,0,10,8.0,0.350000,68,64,73,73
2,0,35,45,8,9,5,16,0.312500,0.666644,0.499988,...,0.555549,1,0,10,8.5,0.361111,68,71,62,79
3,1,50,44,9,10,6,19,0.315789,0.749981,0.999967,...,0.599994,1,0,6,9.5,0.533333,74,84,70,84
4,0,37,43,10,10,9,20,0.450000,0.799984,0.799984,...,0.899991,1,1,6,10.0,0.684211,88,78,85,96


In [ ]:
## use tf-idf

from sklearn.feature_extraction.text import TfidfVectorizer
# join the texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

tfidf = TfidfVectorizer(max_features = 3000)
q1_arr, q2_arr = np.vsplit(tfidf.fit_transform(questions).toarray(), 2)

In [ ]:
temp_df1 = pd.DataFrame(q1_arr, index = ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index = ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2],axis =1)

In [ ]:
final_df = pd.concat([final_df, temp_df], axis = 1)
final_df.shape


(30000, 6023)

In [ ]:
final_df.head()

,is_duplicate,q1_len,q2_len,q1_word,q2_word,word_common,total_words,word_share,cwc_min,cwc_max,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0,48,49,9,9,3,18,0.166667,0.199996,0.249994,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,39,49,7,9,3,16,0.187500,0.599988,0.499992,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,35,45,8,9,5,16,0.312500,0.666644,0.499988,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,50,44,9,10,6,19,0.315789,0.749981,0.999967,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,37,43,10,10,9,20,0.450000,0.799984,0.799984,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
x = final_df.iloc[:,1:].values
y = final_df.iloc[:,0].values

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 42)

In [ ]:
## Logistic regression
model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nconfusion matrix:\n", confusion_matrix(y_test,y_pred))

Accuracy: 0.6995

Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.65      0.68      2973
           1       0.68      0.75      0.72      3027

    accuracy                           0.70      6000
   macro avg       0.70      0.70      0.70      6000
weighted avg       0.70      0.70      0.70      6000


confusion matrix:
 [[1925 1048]
 [ 755 2272]]


In [ ]:
# Decision tree  classifier
model = DecisionTreeClassifier(criterion='gini', random_state=42)  # or use 'entropy'
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nconfusion matrix:\n", confusion_matrix(y_test,y_pred))

Accuracy: 0.7121666666666666

Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.70      0.71      2973
           1       0.71      0.73      0.72      3027

    accuracy                           0.71      6000
   macro avg       0.71      0.71      0.71      6000
weighted avg       0.71      0.71      0.71      6000


confusion matrix:
 [[2068  905]
 [ 822 2205]]


In [ ]:
# ranfom forest classifier
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nconfusion matrix:\n", confusion_matrix(y_test,y_pred))

Accuracy: 0.7906666666666666

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.69      0.77      2973
           1       0.75      0.89      0.81      3027

    accuracy                           0.79      6000
   macro avg       0.80      0.79      0.79      6000
weighted avg       0.80      0.79      0.79      6000


confusion matrix:
 [[2057  916]
 [ 340 2687]]


In [ ]:
# Xtreme gradient boost classifier
xgb = XGBClassifier()
xgb.fit(x_train, y_train)
y_pred = xgb.predict(x_test)
accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nconfusion matrix:\n", confusion_matrix(y_test,y_pred))

Accuracy: 0.7891666666666667

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.71      0.77      2973
           1       0.75      0.87      0.81      3027

    accuracy                           0.79      6000
   macro avg       0.80      0.79      0.79      6000
weighted avg       0.80      0.79      0.79      6000


confusion matrix:
 [[2110  863]
 [ 402 2625]]
